# Notebook 115: 高度な最適化テクニック

## Advanced Optimization Techniques

---

### このノートブックの位置づけ

**Phase 10「最適化手法」** の第6章として、最新の高度な最適化テクニックを学びます。

### 学習目標

1. **Gradient Clipping** を理解する
2. **SAM（Sharpness-Aware Minimization）** を学ぶ
3. **LARS/LAMB** を理解する
4. **Lookahead** オプティマイザを学ぶ
5. **Gradient Accumulation** を理解する

### 前提知識

- Notebook 110-114 の内容

---

## 目次

1. [Gradient Clipping](#1-gradient-clipping)
2. [SAM（Sharpness-Aware Minimization）](#2-samsharpness-aware-minimization)
3. [LARS（Layer-wise Adaptive Rate Scaling）](#3-larslayer-wise-adaptive-rate-scaling)
4. [LAMB（Layer-wise Adaptive Moments for Batch training）](#4-lamblayer-wise-adaptive-moments-for-batch-training)
5. [Lookahead Optimizer](#5-lookahead-optimizer)
6. [Gradient Accumulation](#6-gradient-accumulation)
7. [混合精度学習](#7-混合精度学習)
8. [演習問題](#8-演習問題)
9. [まとめ](#9-まとめ)

In [ ]:
# 環境セットアップ
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = ['Hiragino Sans', 'Arial Unicode MS', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

np.random.seed(42)

print("環境セットアップ完了")

---

## 1. Gradient Clipping

### 1.1 問題: 勾配爆発

RNNなどでは勾配が非常に大きくなることがあり、学習が不安定になります。

### 1.2 解決策

勾配のノルムが閾値を超えた場合、スケーリングします。

$$
g \leftarrow \begin{cases}
g & \text{if } \|g\| \le \tau \\
\frac{\tau}{\|g\|} g & \text{if } \|g\| > \tau
\end{cases}
$$

In [ ]:
def clip_grad_norm(gradients, max_norm=1.0):
    """
    勾配クリッピング（ノルムベース）
    
    Args:
        gradients: 勾配のリスト
        max_norm: 最大ノルム
    
    Returns:
        クリッピング後の勾配, 元のノルム
    """
    # 全勾配のノルムを計算
    total_norm = np.sqrt(sum(np.sum(g**2) for g in gradients))
    
    # クリッピング係数
    clip_coef = max_norm / (total_norm + 1e-6)
    
    if clip_coef < 1:
        gradients = [g * clip_coef for g in gradients]
    
    return gradients, total_norm


def clip_grad_value(gradients, clip_value=1.0):
    """
    勾配クリッピング（値ベース）
    各要素を [-clip_value, clip_value] にクリップ
    """
    return [np.clip(g, -clip_value, clip_value) for g in gradients]


# デモンストレーション
np.random.seed(42)

# 大きな勾配をシミュレート
gradients = [np.random.randn(100) * 10]  # ノルム約100

print(f"元の勾配ノルム: {np.linalg.norm(gradients[0]):.2f}")

# ノルムベースクリッピング
clipped_norm, original_norm = clip_grad_norm(gradients.copy(), max_norm=1.0)
print(f"クリッピング後のノルム: {np.linalg.norm(clipped_norm[0]):.2f}")

# 可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(gradients[0], bins=30, alpha=0.7, label='元の勾配')
axes[0].hist(clipped_norm[0], bins=30, alpha=0.7, label='クリッピング後')
axes[0].set_xlabel('勾配値')
axes[0].set_ylabel('頻度')
axes[0].set_title('ノルムベースクリッピング')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 値ベースクリッピング
clipped_value = clip_grad_value(gradients.copy(), clip_value=5.0)
axes[1].hist(gradients[0], bins=30, alpha=0.7, label='元の勾配')
axes[1].hist(clipped_value[0], bins=30, alpha=0.7, label='クリッピング後')
axes[1].set_xlabel('勾配値')
axes[1].set_ylabel('頻度')
axes[1].set_title('値ベースクリッピング')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 2. SAM（Sharpness-Aware Minimization）

### 2.1 アイデア

**損失曲面の「鋭さ」** を考慮した最適化。平坦な極小点に収束することで汎化性能を向上。

### 2.2 数式

$$
\min_\theta \max_{\|\epsilon\| \le \rho} L(\theta + \epsilon)
$$

2ステップの近似：
1. 摂動を計算: $\epsilon = \rho \cdot \nabla L / \|\nabla L\|$
2. 摂動位置での勾配で更新: $\theta \leftarrow \theta - \eta \nabla L(\theta + \epsilon)$

In [ ]:
class SAM:
    """
    Sharpness-Aware Minimization
    
    1. 現在の勾配でε方向を計算
    2. θ+ε位置での勾配で更新
    """
    
    def __init__(self, base_optimizer, rho=0.05):
        """
        Args:
            base_optimizer: ベースのオプティマイザ（SGD, Adam等）
            rho: 摂動の大きさ
        """
        self.base_optimizer = base_optimizer
        self.rho = rho
    
    def first_step(self, params, gradients):
        """摂動を計算してパラメータに適用"""
        # 勾配ノルム
        grad_norm = np.sqrt(sum(np.sum(g**2) for g in gradients))
        
        # 摂動を計算
        self.e_w = [self.rho * g / (grad_norm + 1e-12) for g in gradients]
        
        # パラメータに摂動を加える
        for p, e in zip(params, self.e_w):
            p += e
    
    def second_step(self, params, gradients):
        """摂動を元に戻し、ベースオプティマイザで更新"""
        # 摂動を元に戻す
        for p, e in zip(params, self.e_w):
            p -= e
        
        # ベースオプティマイザで更新
        self.base_optimizer.update(params, gradients)


# SAMの効果を可視化

def sharp_loss(x, y):
    """鋭い極小点を持つ損失関数"""
    return (x - 1)**2 + 10 * np.sin(5 * x)**2 + (y - 1)**2 + 10 * np.sin(5 * y)**2

def sharp_loss_grad(x, y):
    dx = 2 * (x - 1) + 100 * np.sin(5 * x) * np.cos(5 * x)
    dy = 2 * (y - 1) + 100 * np.sin(5 * y) * np.cos(5 * y)
    return np.array([dx, dy])


# 損失曲面の可視化
x = np.linspace(-1, 3, 200)
y = np.linspace(-1, 3, 200)
X, Y = np.meshgrid(x, y)
Z = sharp_loss(X, Y)

fig, ax = plt.subplots(figsize=(10, 8))
contour = ax.contour(X, Y, Z, levels=50, cmap='viridis')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('鋭い極小点を持つ損失曲面')
plt.colorbar(contour)

plt.tight_layout()
plt.show()

print("【SAMの利点】")
print("- 平坦な極小点に収束しやすい")
print("- 汎化性能が向上")
print("- 計算コストは約2倍（2回の順伝播/逆伝播）")

---

## 3. LARS（Layer-wise Adaptive Rate Scaling）

### 3.1 アイデア

大きなバッチサイズでの学習を可能にするため、**層ごとに学習率を調整** します。

### 3.2 数式

$$
\lambda^l = \eta \cdot \frac{\|\theta^l\|}{\|\nabla L^l\| + \beta \|\theta^l\|}
$$

各層の重みと勾配のノルムの比率で学習率をスケーリング。

In [ ]:
class LARS:
    """
    Layer-wise Adaptive Rate Scaling
    
    大バッチ学習のための層ごとの学習率スケーリング
    """
    
    def __init__(self, lr=0.1, momentum=0.9, weight_decay=1e-4, trust_coef=0.001):
        self.lr = lr
        self.momentum = momentum
        self.weight_decay = weight_decay
        self.trust_coef = trust_coef
        self.velocities = {}
    
    def update(self, layer_params, layer_grads, layer_names):
        """
        Args:
            layer_params: 層ごとのパラメータのリスト
            layer_grads: 層ごとの勾配のリスト
            layer_names: 層の名前のリスト
        """
        for name, param, grad in zip(layer_names, layer_params, layer_grads):
            # 層ごとのノルム
            param_norm = np.linalg.norm(param)
            grad_norm = np.linalg.norm(grad)
            
            # 信頼比率（trust ratio）
            if param_norm > 0 and grad_norm > 0:
                local_lr = self.trust_coef * param_norm / (grad_norm + self.weight_decay * param_norm)
            else:
                local_lr = 1.0
            
            # Weight decay を加える
            grad = grad + self.weight_decay * param
            
            # Momentum
            if name not in self.velocities:
                self.velocities[name] = np.zeros_like(param)
            
            self.velocities[name] = self.momentum * self.velocities[name] + self.lr * local_lr * grad
            
            # パラメータ更新
            param -= self.velocities[name]


print("【LARSの用途】")
print("- 大規模なバッチサイズ（8192, 32768等）での学習")
print("- ImageNet学習の高速化（30分で学習完了等）")
print("")
print("【なぜ大バッチで問題が起きるか】")
print("- 学習率を線形にスケールすると発散")
print("- 層によって最適な学習率が異なる")
print("- LARSは各層の状態に応じて学習率を調整")

---

## 4. LAMB（Layer-wise Adaptive Moments for Batch training）

### 4.1 アイデア

LARSをAdamに拡張したもの。BERTなどの大規模モデルの学習に使用。

### 4.2 数式

$$
r = \frac{\hat{m}}{\sqrt{\hat{v}} + \epsilon} + \lambda \theta
$$
$$
\theta \leftarrow \theta - \eta \cdot \frac{\|\theta\|}{\|r\|} \cdot r
$$

In [ ]:
class LAMB:
    """
    Layer-wise Adaptive Moments for Batch training
    
    LARS + Adam
    """
    
    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999, eps=1e-6, weight_decay=0.01):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self.weight_decay = weight_decay
        self.m = {}
        self.v = {}
        self.t = 0
    
    def update(self, layer_params, layer_grads, layer_names):
        self.t += 1
        
        for name, param, grad in zip(layer_names, layer_params, layer_grads):
            # Adamのモーメント
            if name not in self.m:
                self.m[name] = np.zeros_like(param)
                self.v[name] = np.zeros_like(param)
            
            self.m[name] = self.beta1 * self.m[name] + (1 - self.beta1) * grad
            self.v[name] = self.beta2 * self.v[name] + (1 - self.beta2) * grad ** 2
            
            m_hat = self.m[name] / (1 - self.beta1 ** self.t)
            v_hat = self.v[name] / (1 - self.beta2 ** self.t)
            
            # Adam更新 + Weight Decay
            r = m_hat / (np.sqrt(v_hat) + self.eps) + self.weight_decay * param
            
            # 層ごとの信頼比率
            param_norm = np.linalg.norm(param)
            r_norm = np.linalg.norm(r)
            
            if param_norm > 0 and r_norm > 0:
                trust_ratio = param_norm / r_norm
            else:
                trust_ratio = 1.0
            
            # パラメータ更新
            param -= self.lr * trust_ratio * r


print("【LAMBの用途】")
print("- BERT, GPTなどの大規模言語モデルの学習")
print("- バッチサイズ 64K 以上での学習")
print("")
print("【LARS vs LAMB】")
print("- LARS: SGD + Momentum ベース → 画像分類向け")
print("- LAMB: Adam ベース → NLP/Transformer向け")

---

## 5. Lookahead Optimizer

### 5.1 アイデア

「速い重み」と「遅い重み」の2つを維持し、定期的に補間することで学習を安定化。

### 5.2 アルゴリズム

1. 内部ループ: ベースオプティマイザでk回更新
2. 外部ループ: 遅い重みを速い重みの方向に補間

$$
\phi_{t+1} = \phi_t + \alpha (\theta_{t,k} - \phi_t)
$$

In [ ]:
class Lookahead:
    """
    Lookahead Optimizer
    
    速い重みθと遅い重みφを維持
    """
    
    def __init__(self, base_optimizer, k=5, alpha=0.5):
        """
        Args:
            base_optimizer: 内部ループのオプティマイザ
            k: 内部ループの回数
            alpha: 補間係数
        """
        self.base_optimizer = base_optimizer
        self.k = k
        self.alpha = alpha
        self.step_counter = 0
        self.slow_params = None
    
    def update(self, params, gradients):
        # 初期化
        if self.slow_params is None:
            self.slow_params = [p.copy() for p in params]
        
        # ベースオプティマイザで更新（内部ループ）
        self.base_optimizer.update(params, gradients)
        
        self.step_counter += 1
        
        # k回ごとに遅い重みを更新（外部ループ）
        if self.step_counter % self.k == 0:
            for slow_p, fast_p in zip(self.slow_params, params):
                # 遅い重みを速い重みの方向に補間
                slow_p += self.alpha * (fast_p - slow_p)
                # 速い重みを遅い重みにリセット
                fast_p[:] = slow_p


# Lookaheadの効果を可視化

def noisy_loss(x, y):
    return (x - 2)**2 + (y - 2)**2 + 0.5 * np.sin(10 * x) * np.sin(10 * y)

def noisy_loss_grad(x, y):
    dx = 2 * (x - 2) + 5 * np.cos(10 * x) * np.sin(10 * y)
    dy = 2 * (y - 2) + 5 * np.sin(10 * x) * np.cos(10 * y)
    return np.array([dx, dy])


# 等高線
x = np.linspace(-1, 5, 200)
y = np.linspace(-1, 5, 200)
X, Y = np.meshgrid(x, y)
Z = noisy_loss(X, Y)

fig, ax = plt.subplots(figsize=(10, 8))
contour = ax.contour(X, Y, Z, levels=30, cmap='viridis', alpha=0.7)
ax.scatter([2], [2], color='red', s=100, marker='*', zorder=5, label='最適点')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('ノイズのある損失曲面')
ax.legend()

plt.tight_layout()
plt.show()

print("【Lookaheadの利点】")
print("- 学習の安定化")
print("- 過度な振動の抑制")
print("- 任意のベースオプティマイザと組み合わせ可能")

---

## 6. Gradient Accumulation

### 6.1 アイデア

メモリ制約がある場合、小さなバッチの勾配を複数回累積してから更新。

→ 実効的なバッチサイズを大きくする

In [ ]:
class GradientAccumulator:
    """
    勾配累積
    
    小さなバッチの勾配を累積し、一定回数後に更新
    """
    
    def __init__(self, optimizer, accumulation_steps=4):
        self.optimizer = optimizer
        self.accumulation_steps = accumulation_steps
        self.accumulated_grads = None
        self.step_counter = 0
    
    def step(self, params, gradients):
        # 勾配を累積
        if self.accumulated_grads is None:
            self.accumulated_grads = [np.zeros_like(g) for g in gradients]
        
        for acc_g, g in zip(self.accumulated_grads, gradients):
            acc_g += g / self.accumulation_steps
        
        self.step_counter += 1
        
        # accumulation_steps回ごとに実際に更新
        if self.step_counter % self.accumulation_steps == 0:
            self.optimizer.update(params, self.accumulated_grads)
            self.accumulated_grads = None
            return True  # 更新が行われた
        
        return False  # 累積中


print("【Gradient Accumulationの使い方】")
print("")
print("例: 目標バッチサイズ 256、GPU メモリで 64 しか入らない場合")
print("")
print("accumulation_steps = 256 // 64 = 4")
print("")
print("for i, (x, y) in enumerate(dataloader):  # batch_size=64")
print("    loss = model(x, y)")
print("    loss.backward()  # 勾配を計算")
print("    ")
print("    if (i + 1) % 4 == 0:  # 4回ごとに更新")
print("        optimizer.step()")
print("        optimizer.zero_grad()")
print("")
print("→ 実効バッチサイズ: 64 × 4 = 256")

---

## 7. 混合精度学習

### 7.1 アイデア

計算にFP16（半精度）を使用し、勾配累積やパラメータ更新にはFP32を使用。

→ メモリ削減と計算高速化

In [ ]:
print("【混合精度学習のメリット】")
print("")
print("1. メモリ使用量の削減")
print("   - FP16は FP32 の半分のメモリ")
print("   - より大きなバッチサイズが可能")
print("")
print("2. 計算速度の向上")
print("   - Tensor Cores（NVIDIA GPU）による高速化")
print("   - 最大2-3倍のスピードアップ")
print("")
print("3. 注意点")
print("   - 勾配のアンダーフロー → Loss Scaling で対処")
print("   - 精度の損失 → マスターウェイトをFP32で維持")
print("")
print("-" * 50)
print("")
print("【Loss Scaling】")
print("")
print("1. 損失を大きな値でスケール: loss_scaled = loss * scale")
print("2. 逆伝播")
print("3. 勾配を元に戻す: grad = grad / scale")
print("4. 更新")
print("")
print("→ 小さな勾配がアンダーフローするのを防ぐ")

---

## 8. 演習問題

### 演習 8.1: SAMの実装と比較

SGDとSAMを比較し、SAMが平坦な極小点に収束することを確認してください。

In [ ]:
# 演習 8.1: 解答欄

# TODO: SAMの完全な実装とテスト

pass

### 演習 8.2: Lookahead + Adam (Ranger)

Lookahead と Adam を組み合わせた「Ranger」オプティマイザを実装してください。

In [ ]:
# 演習 8.2: 解答欄

# TODO: Ranger = Lookahead(RAdam) の実装

pass

---

## 9. まとめ

### このノートブックで学んだこと

1. **Gradient Clipping**: 勾配爆発を防ぐ

2. **SAM**: 平坦な極小点への収束で汎化性能向上

3. **LARS/LAMB**: 大バッチ学習のための層ごとの学習率調整

4. **Lookahead**: 速い重みと遅い重みの組み合わせで安定化

5. **Gradient Accumulation**: 実効バッチサイズの拡大

6. **混合精度学習**: メモリ効率と速度の向上

### 高度なテクニックの選択指針

| テクニック | 用途 |
|-----------|------|
| Gradient Clipping | RNN、不安定な学習 |
| SAM | 汎化性能が重要な場合 |
| LARS | 大バッチ画像分類 |
| LAMB | 大バッチNLP/Transformer |
| Lookahead | 学習の安定化 |
| Gradient Accumulation | メモリ制約下での大バッチ |
| 混合精度 | 高速化、メモリ効率 |

---

## 参考文献

1. Foret, P., et al. (2021). Sharpness-aware minimization for efficiently improving generalization. *ICLR*.
2. You, Y., et al. (2017). Large batch training of convolutional networks. *arXiv:1708.03888*.
3. You, Y., et al. (2020). Large batch optimization for deep learning: Training BERT in 76 minutes. *ICLR*.
4. Zhang, M., et al. (2019). Lookahead optimizer: k steps forward, 1 step back. *NeurIPS*.
5. Micikevicius, P., et al. (2018). Mixed precision training. *ICLR*.